In [11]:
import pandas as pd
import ast
import numpy as np
from detecto import core, utils, visualize
from matplotlib import pyplot as plt
from tqdm import tqdm
import cv2

In [2]:
data = pd.read_csv('data/train.csv')
fn, w, h, cl, boxes, imgid = [], [], [], [], [], []
counter = 0
for i in range(300):
    idx = int(data['ImageID'].iloc[i])
    bbox = ast.literal_eval(data['bboxes'].iloc[i])
    for box in bbox:
        fn.append(f'{idx}.jpg')
        w.append(512)
        h.append(512)
        cl.append('debris')
        boxes.append(box)
        imgid.append(counter)
        counter += 1
boxes = np.array(boxes).T
pd.DataFrame(
    {'filename':fn, 
     'width' : w, 
     'height' : h, 
     'class' : cl,
     'xmin' : boxes[0],
     'ymin' : boxes[2],
     'xmax' : boxes[1],
     'ymax':boxes[3],
     'image_id':imgid
    }
).to_csv('data/labels.csv',index = False)

In [3]:
dataset = core.Dataset('data/labels.csv','data/train')
model = core.Model(['debris'],pretrained = False)
m = model.fit(dataset,epochs=10, learning_rate=0.001, lr_step_size=5,verbose = True)

  0%|                                                                                         | 0/1137 [00:00<?, ?it/s]

Epoch 1 of 1
Begin iterating over training dataset


100%|██████████████████████████████████████████████████████████████████████████████| 1137/1137 [03:02<00:00,  6.24it/s]


In [4]:
out = []
for i in tqdm(range(5000)):
    image = utils.read_image(f'data/test/{i}.jpg')
    labels, boxes, scores = model.predict(image)
    bb = []
    for u,box in enumerate(boxes):
        box = box.cpu().tolist()
        bb.append([box[0],box[2],box[1],box[3],float(scores[u]+0.2 if scores[u]+0.2 <= 1 else 1)])
    out.append(bb)

100%|██████████████████████████████████████████████████████████████████████████████| 5000/5000 [06:34<00:00, 12.67it/s]


In [7]:
def draw_bboxes(img, bboxes, train,color=(255, 0, 0), thickness=1):
    for u, bbox in enumerate(bboxes):
        # if [x1, y1, x2, y2]
        img = cv2.rectangle(img, (int(bbox[0]),int(bbox[2])), (int(bbox[1]),int(bbox[3])), color, thickness)
    return img

In [16]:
num = 5
i = utils.read_image(f'data/test/{num}.jpg')
#i = cv2.flip(i,0)
image = draw_bboxes(i,out[num],True)
#image = draw_bboxes(image,t[num],False,(0,255,0))
plt.clf()
plt.figure(figsize=(10,10))
plt.imshow(image)

<Figure size 720x720 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 720x720 with 0 Axes>

In [13]:
df = pd.read_csv('data/sample_submission.csv')
df.bboxes = out
df.to_csv('submission.csv',index = False)